In [1]:
# ปรับแต่งค่าการทำงานของ Spark
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("KafkaSubscribe").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1000m").\
        config("spark.executor.cores", "2").\
        config("spark.cores.max", "6").\
        config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0").\
        getOrCreate()



Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-18a83cdf-80dc-4184-8c8a-7be6a7b5dde9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 652ms :: artifacts dl 15

In [2]:
rawMetadata_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "10.128.0.12:9092") \
  .option("subscribe", "quickstart-events") \
  .option("group.id", "Aekanun-Spark-App") \
  .load()

In [3]:
from pyspark.sql import functions as sparkf

In [4]:
rawMetadata_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
# เลือกเฉพาะคอลัมน์ที่ต้องการ
onlyMetadata_df = rawMetadata_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [6]:
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col

# Main schema
schema = StructType([
    StructField("locationId", StringType()),
    StructField("location", StringType()),
    StructField("parameter", StringType()),
    StructField("value", StringType()),
    StructField("date", StringType()),
    StructField("unit", StringType()),
    StructField("coordinates", StringType()),
    StructField("country", StringType()),
    StructField("city", StringType()),
    StructField("isMobile", StringType()),
    StructField("isAnalysis", StringType()),
    StructField("entity", StringType()),
    StructField("sensorType", StringType())
])

# Nested 'date' and 'coordinates' schema
date_schema = StructType([
    StructField("utc", StringType()),
    StructField("local", StringType())
])

coordinates_schema = StructType([
    StructField("latitude", DoubleType()),
    StructField("longitude", DoubleType())
])

# Parse the JSON string column and convert it to a struct.
parsedData_df = onlyMetadata_df.withColumn("data", from_json("value", schema))

In [7]:
parsedData_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- locationId: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- parameter: string (nullable = true)
 |    |-- value: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- unit: string (nullable = true)
 |    |-- coordinates: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- isMobile: string (nullable = true)
 |    |-- isAnalysis: string (nullable = true)
 |    |-- entity: string (nullable = true)
 |    |-- sensorType: string (nullable = true)



In [8]:
# Select only the nested fields
unNested_df = parsedData_df.select("key", "data.*")

In [9]:
unNested_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- date: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)



In [10]:
# Parse the JSON string column and convert it to a struct.
unNested_df.withColumn("data", from_json("value", schema))\
.withColumn("date", from_json("date", date_schema))\
.withColumn("coordinates", from_json("coordinates", coordinates_schema))\
.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- date: struct (nullable = true)
 |    |-- utc: string (nullable = true)
 |    |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- locationId: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- parameter: string (nullable = true)
 |    |-- value: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- unit: st

In [11]:
extractedDateLatLong_df = unNested_df.withColumn("data", from_json("value", schema))\
.withColumn("date", from_json("date", date_schema))\
.withColumn("coordinates", from_json("coordinates", coordinates_schema))\
.select('key',
 'locationId',
 'location',
 'parameter',
 'value',
 'date.*',
 'unit',
 'coordinates.*',
 'country',
 'city',
 'isMobile',
 'isAnalysis',
 'entity',
 'sensorType')

In [12]:
extractedDateLatLong_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- utc: string (nullable = true)
 |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)



In [13]:
final_df = extractedDateLatLong_df\
.withColumn('sourceUnixSTP',sparkf.unix_timestamp(sparkf.col('utc'), "yyyy-MM-dd'T'HH:mm:ssXXX"))

In [14]:
final_df.columns

['key',
 'locationId',
 'location',
 'parameter',
 'value',
 'utc',
 'local',
 'unit',
 'latitude',
 'longitude',
 'country',
 'city',
 'isMobile',
 'isAnalysis',
 'entity',
 'sensorType',
 'sourceUnixSTP']

In [15]:
final_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- utc: string (nullable = true)
 |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)
 |-- sourceUnixSTP: long (nullable = true)



In [16]:
#agg_df = final_df.groupBy('value').agg(sparkf.max(sparkf.col('value')))

In [ ]:
from pyspark.sql import functions as F

def max_value(df, epoch_id):
    df = df.withColumn("value", df["value"].cast("float"))
    max_value = df.agg(F.max(df.value)).collect()[0][0]
    print(f"Batch: {epoch_id}, Max value: {max_value}")

query = final_df.writeStream \
    .foreachBatch(max_value) \
    .outputMode("update") \
    .trigger(processingTime='5 seconds') \
    .start()

query.awaitTermination()

23/06/17 11:26:54 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6a3ea633-4418-47d7-a812-7ce42e36c55e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/06/17 11:27:01 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 6380 milliseconds


Batch: 0, Max value: None


Batch: 1, Max value: 25.0


23/06/17 11:27:12 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 6482 milliseconds


Batch: 2, Max value: 24.0
Batch: 3, Max value: 17.0
Batch: 4, Max value: 15.0
Batch: 5, Max value: 18.0
Batch: 6, Max value: 18.0
Batch: 7, Max value: 15.0
Batch: 8, Max value: 17.0
Batch: 9, Max value: 20.0
Batch: 10, Max value: 16.0
Batch: 11, Max value: 17.0
Batch: 12, Max value: 21.0
Batch: 13, Max value: 19.0
Batch: 14, Max value: 15.0
Batch: 15, Max value: 19.0
Batch: 16, Max value: 16.0
Batch: 17, Max value: 17.0
Batch: 18, Max value: 16.0
Batch: 19, Max value: 14.0
Batch: 20, Max value: 24.0
Batch: 21, Max value: 22.0
Batch: 22, Max value: 18.0
Batch: 23, Max value: 21.0
Batch: 24, Max value: 20.0
Batch: 25, Max value: 21.0
Batch: 26, Max value: 24.0
Batch: 27, Max value: 25.0
Batch: 28, Max value: 35.0
Batch: 29, Max value: 32.0
Batch: 30, Max value: 32.0
Batch: 31, Max value: 23.0
Batch: 32, Max value: 17.0
Batch: 33, Max value: 15.0
Batch: 34, Max value: 22.0
Batch: 35, Max value: 21.0
Batch: 36, Max value: 14.0
Batch: 37, Max value: 26.0
Batch: 38, Max value: 26.0
Batch: 39

In [ ]:
query.stop()